<a href="https://colab.research.google.com/github/cerr/octave-colab/blob/master/CT_Heart_SubStructures_Dropbox_IDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Heart SubStructure CT Segmentation

### Download software and set up dependencies
CERR, GNU Octave, condapack Python files, heart DeepLab segmentation network

In [9]:
#Dropbox download URL
condaenv_gz = ''
heartdeeplab_gz = ''
sampledata_mat = ''

In [1]:
import os, subprocess, urllib.request, json
from glob import glob

In [2]:
%%capture
! apt-get update
! apt-get install libgraphicsmagick++1-dev libsuitesparse-dev libqrupdate1 libfftw3-3 gnuplot zsh
! pip install oct2py octave_kernel

In [5]:
# download compiled octave package (latest)
def get_CERRoctave(root_path):
  os.chdir(root_path)
  with urllib.request.urlopen("https://api.github.com/repos/cerr/octave-colab/releases/latest") as url:
      data = json.loads(url.read().decode())
  fname = data['assets'][0]['name']
  requrl = data['assets'][0]['browser_download_url']
  urllib.request.urlretrieve(requrl, fname)
  # unzip, identify octave folder name
  !tar xf {fname}
  top_folder = !tar tf {fname} | head -1
  octave_folder = top_folder[0][:-1]
  octave_path = os.path.join(root_path,octave_folder)
  !git clone https://github.com/cerr/CERR.git
  !cd CERR && git checkout octave_dev
  cerr_path = os.path.join(root_path,'CERR')
  return octave_path, cerr_path


octave_path,cerr_path = get_CERRoctave('/content')
os.environ['OCTAVE_EXECUTABLE'] = octave_path + '/bin/octave-cli'
os.environ['PATH'] = octave_path + '/bin:' + os.environ['PATH']

In [ ]:
import oct2py
import octave_kernel
from oct2py import octave
%load_ext oct2py.ipython

In [ ]:
%%capture
!mkdir /content/{condaenv,data}
!wget {condaenv_gz} -O /content/condaenv/CT_Heart_DeepLab_hdl35_linux.tar.gz && \
wget {heartdeeplab_gz} -O /content/condaenv/CT_Heart_DeepLab.tar.gz && \
wget {sampledata_mat} -O /content/data/sample_planC.mat

In [ ]:
%%capture
gz_list = glob('/content/condaenv/*.tar.gz')
for g in gz_list:
  !tar xvf {g} --directory /content/condaenv

Initialize Octave with oct2py bridge

In [ ]:
%%octave

pkg load image
pkg load io
pkg load statistics

warning('off','all')

In [ ]:
cerrPath = r'/content/CERR'
octave.addpath(cerrPath)
octave.addToPath2(cerrPath)

In [ ]:
!rm -rf /content/sessions

Run segmentation routine

In [ ]:
%%octave

planCFileName = '/content/data/sample_planC.mat';

sessionPath = '/content/sessions';
cmdFlag = 'condaEnv';
condaEnvName = '/content/condaenv';
algorithm = 'CT_HeartSubStructures_DeepLab';
wrapperFunction = '/content/condaenv/CT_Heart_DeepLab/model_wrapper/CT_HeartSubStructures_DeepLab/runSegHeartSubStructures.py';
batchSize = 1;

planC = loadPlanC(planCFileName);

[activate_cmd,run_cmd,userOptS,outFile,scanNumV,planC] = prepDataForSeg(planC,sessionPath,algorithm,cmdFlag,condaEnvName, {wrapperFunction});
[outStrListC,labelMapS] = getAutosegStructnames(userOptS);

In [ ]:
%octave_pull outFile

In [ ]:
!chmod 777 {outFile}

In [ ]:
q = subprocess.Popen(outFile,stdout=subprocess.PIPE,stderr=subprocess.PIPE)
s=q.communicate()
print(s)

In [ ]:
%%octave
planC = processAndImportSeg(planC,1,sessionPath,userOptS);

Visualization

In [ ]:
%%octave

%Get scan array
indexS = planC{end};
scanNum = 1;
ctOffset = planC{indexS.scan}(scanNum).scanInfo(1).CTOffset;
scanArray = single(getScanArray(scanNum,planC)) - ctOffset;

%Get structure labels & masks
numStructs = length(planC{indexS.structures});
structNameC = {planC{indexS.structures}.structureName};
strNameC = outStrListC;
for strNum = 1:length(strNameC)
    strx = strNameC{strNum};
    idx = getMatchingIndex(strx,structNameC,'EXACT');
    mask3M = getStrMask(idx, planC);
    maskC{strNum} = mask3M;
end

In [ ]:
%octave_pull maskC strNameC scanArray

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
import ipywidgets as widgets

dx, dy = 1, 1

x = np.arange(0, 255, dx)
y = np.arange(0, 255, dy)
extent = np.min(x), np.max(x), np.min(y), np.max(y)

clear_output(wait=True)

def window_image(image, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    window_image = image.copy()
    window_image[window_image < img_min] = img_min
    window_image[window_image > img_max] = img_max
    
    return window_image

def show_axial_slice(slcNum):
    clear_output(wait=True)
    print('Slice '+str(slcNum))
    if 'fig' in locals():
        fig.remove()
    fig, ax = plt.subplots()
    window_center = 45
    window_width = 125
    windowed_img = window_image(scanArray[:,:,slcNum-1],
                                window_center,window_width)
    im1 = ax.imshow(windowed_img, cmap=plt.cm.gray, alpha=1,
                    interpolation='nearest', extent=extent)
    
    cmaps = [plt.cm.Oranges,plt.cm.Oranges,plt.cm.Blues,plt.cm.Blues, \
             plt.cm.Purples,plt.cm.Greens]

    for maskNum in range(0,6,1):
        mask_cmap = cmaps[maskNum]
        mask_cmap.set_under('k', alpha=0)
        im2 = ax.imshow(maskC[0,maskNum][:,:,slcNum-1], 
                        cmap=mask_cmap, alpha=.8, extent=extent,
                        interpolation='none', clim=[0.5, 1])        
    plt.show()

slice_slider = widgets.IntSlider(min=1,max=79,step=1)
outputSlc = widgets.Output()

display(slice_slider, outputSlc)

def update_slice(change):
    with outputSlc:
        show_axial_slice(change['new'])

slice_slider.observe(update_slice, names='value')
